## Inverse Kinematics for 2-Link Planar Robot

In this notebook:
- Given a target `(x, y)`, compute joint angles `(theta1, theta2)` to reach it. (Using Analytic Solution)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# We know our robot links length
L1 = 1
L2 = 1

# But now consider we have a desired end-effector position, and we don't know thetas...
x_target, y_target = 0.87, 1.5

# We know the robot config, so we have our transformation matrix
def T2D(theta, L):
    return np.array([
        [np.cos(theta), -np.sin(theta), L*np.cos(theta)],
        [np.sin(theta),  np.cos(theta), L*np.sin(theta)],
        [0, 0, 1]
    ])

In [ ]:
def inverse_kinematics(x, y, L1, L2):
    r = np.sqrt(x**2 + y**2) # Compute distance from base to point
    
    # Check if the point is reachable (Within Robot Workspace)
    if r > (L1 + L2):
        raise ValueError("Target is out of reach")
    
    # Law of cosines for theta2 (a2 = b2 + c2 - 2bc*cosA --> cosA = (b2 + c2 - a2)/2bc)
    cos_theta2 = (x**2 + y**2 - L1**2 - L2**2) / (2*L1*L2)
    theta2 = np.arccos(cos_theta2)  # toggle sign here to see redundancy in action...
    
    # Law of cosines for theta1
    k1 = L1 + L2*np.cos(theta2)
    k2 = L2*np.sin(theta2)
    theta1 = np.arctan2(y, x) - np.arctan2(k2, k1)
    
    return theta1, theta2

### Test Inverse Kinematics

In [ ]:
theta1_ik, theta2_ik = inverse_kinematics(x_target, y_target, L1, L2)

print(f"Computed angles (radians): theta1={theta1_ik:.2f}, theta2={theta2_ik:.2f}")
print(f"Computed angles (degrees): theta1={np.degrees(theta1_ik):.1f}, theta2={np.degrees(theta2_ik):.1f}")

### Visualize Robot at Target Using IK

In [ ]:
# Compute forward kinematics with IK solution
BASE_T_JOINT1_ik = T2D(theta1_ik, L1)
JOINT1_T_JOINT2_ik = T2D(theta2_ik, L2)
BASE_T_JOINT2_ik = BASE_T_JOINT1_ik @ JOINT1_T_JOINT2_ik

x0, y0 = 0, 0
x1, y1 = BASE_T_JOINT1_ik[0,2], BASE_T_JOINT1_ik[1,2]
x2, y2 = BASE_T_JOINT2_ik[0,2], BASE_T_JOINT2_ik[1,2]

plt.figure(figsize=(5,5))
plt.plot([x0, x1, x2], [y0, y1, y2], '-o', linewidth=4, markersize=10, label='Robot')
plt.plot(x_target, y_target, 'rx', markersize=12, label='Target')
plt.xlim(-1, 5)
plt.ylim(-1, 4)
plt.title('2-Link Planar Robot Inverse Kinematics')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.legend()
plt.show()

### Notes:
- The IK solution may have two configurations (elbow-up or elbow-down), called ***redundancy***. 
Here we calculate one, but this demonstrates 1 weakness with IK calculations that might appear.
- Path planning algorithms using AI need to be robust against this (Use other criteria).